# <span style="font-family:Courier New; color:#CCCCCC">**Pràctica 3: CA & Heuristics**</span>

In [ ]:
import numpy as np

## <span style="font-family:Courier New; color:#336666">**Forest Fire CA model**</span>

In [1]:
class Multilayer2DCA:

    def __init__(self, data):
        self._parse(data)
        self._size_x = 100
        self._size_y = 100
        self.state = np.zeros((3, 100, 100), dtype = int)


    def _parse(file):
        pass


    def get_hood_info(self, layer, i, j):
        """
        Method to get the 'layer's states of the (i, j)'s Moore Neighborhood cells
        """

        hood_states = []
        for x in range(-1, 2):
            for y in range(-1, 2):
                if x == 0 and y == 0:
                    continue
                ni = (i + x) if (i + x < self._size_x) and (i + x >= 0) else None # Closed-Boundary Assumption
                nj = (j + y) if (j+y < self._size_y) and (j + y >= 0) else None
                if ni and nj: hood_states.append(self.state[layer, ni, nj])
        return hood_states
    

    def evolve(self):
        """
        Method to evolve bi-dimensional cellular automaton from time (t) to (t + 1)
        """

        self._new_state = np.zeros((self.num_layers, self.size_x, self.size_y), dtype = int)
        self._evolve_humidity()
        self._evolve_fuel()
        self._evolve_fire()
        self.state = self._new_state
    
    
    def _evolve_humidity(self):
        """
        Method to update humidity's layer from time (t) to (t + 1)
        """

        for i in range(self._size_x):
            for j in range(self._size_y):

                cell_humidity = self.state[2, i, j]
                hood_fire = self.get_hood_info(0, i, j)
                self._new_state[2, i, j] = self._humidity_func(cell_humidity, hood_fire)
    

    def _evolve_fuel(self):
        """
        Method to update fuel's layer from time (t) to (t + 1)
        """

        for i in range(self._size_x):
            for j in range(self._size_y):

                cell_fire, cell_fuel, cell_humidity = self.state[:, i, j]
                hood_fire = self.get_hood_info(0, i, j)
                self._new_state[0, i, j] = self._fire_func(cell_fire, cell_fuel, cell_humidity, hood_fire)


    def _evolve_fire(self):
        """
        Method to update fire's layer from time (t) to (t + 1)
        """

        for i in range(self._size_x):
                for j in range(self._size_y):

                    cell_fire, cell_fuel, _ = self.state[:, i, j]  
                    self._new_state[1, i, j] = self._fuel_func(cell_fire, cell_fuel)   
        

    def simulate(self, t):
        """
        Method to carry out a simulation of t units of time

        Return: list with the bi-dimensional states of each generation step
        """

        evolution_steps = [self.state.copy()]
        for _ in range(t):
            self.evolve()
            evolution_steps.append(self.state.copy())
        return evolution_steps
    

    def _fire_func(self, cell_fire, cell_fuel, cell_humidity, hood_fire):
        """
        Method to calculate the next state of a fire's layer cell, combining information from multiple current layers

        Return: -1 if Burned, 0 if UnBurned or 1 if Burning
        """

        nghbrs_burning = sum(1 for state in hood_fire if state == 1)
        if cell_fire == -1: return -1
        elif cell_fire == 0:
            if nghbrs_burning > 0 and cell_fuel > 0 and cell_humidity == 0:
                return 1
            else:
                return 0
        elif cell_fire == 1:
            if cell_fuel > 1: return 1   
            else: return 0
              

    def _fuel_func(self, cell_fire, cell_fuel):
        """
        Method to calculate the next state of a fuel's layer cell, combining information from multiple current layers

        Return: number of hours that a cell can remain in Burning state
        """
        res = cell_fuel - 1 if cell_fire == 1 else cell_fuel
        return res if res > 0 else 0


    def _humidity_func(self, cell_humidity, hood_fire):
        """
        Method to calculate the next state of a humidity's layer cell, combining information from multiple current layers

        Return: number of hours that a cell can delay its Unburned state, when hood_fire is active
        """

        nghbrs_fire = sum(1 for state in hood_fire if state == 1)
        reduce_t = (1.5* np.log(nghbrs_fire)) + 1 if nghbrs_fire > 0 else 0 #Humity reduction caused by active hood fire
        res = cell_humidity - reduce_t
        return res if res > 0 else 0

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def visualize(evolution_steps):
    fig = plt.figure()
    ims = []
    for step in evolution_steps:
        im = plt.imshow(step, cmap='binary', animated=True)
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True, repeat_delay=1000)
    plt.show()